In [1]:
import gym

In [2]:
env=gym.make('MountainCar-v0')

# Lets See Whats this Game is all about first!!!!

In [ ]:
for e in range(20):
    observation=env.reset()
    for t in range(500):
        env.render()
        action=env.action_space.sample()
        observation,reward,done,other_info=env.step(action)
        if done:
            print("Game Over {}/{} High Score {} ".format(e,20,t))
            break
env.close()
print("All 20 episodes over!!!")

## Now , Lets  Train a DQ agent who will complete Game with High Score

In [ ]:
env.action_space.n # so we can use 3 actions, 

In [ ]:
env.observation_space.shape[0] # so we have 2 types of observation that we will feed to the neural network

In [3]:
import numpy as np
import os
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import random
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [4]:
class Agent:
    def __init__(self,action_size,state_size):
        self.action_size=action_size
        self.state_size=state_size
        self.gamma=0.95
        self.epsilon=1.0
        self.epsilon_decay=0.995
        self.epsilon_min=0.01
        self.memory=deque(maxlen=2000)
        self.learning_rate=0.001
        self.model=self._create_model()
    def _create_model(self):
        model=Sequential()
        model.add(Dense(24,input_dim=(self.state_size),activation='relu'))
        model.add(Dense(24,activation='relu'))
        model.add(Dense(self.action_size,activation='linear'))
        model.compile(loss='mse',optimizer=Adam(lr=self.learning_rate))
        return model
    def remember(self,state,action,reward,next_state,done):
        self.memory.append((state,action,reward,next_state,done))
    def act(self,state):
        if np.random.rand()<=self.epsilon:
            return random.randrange(self.action_size)
        return np.argmax(self.model.predict(state)[0])
    def train(self,batch_size=32):
        mini_batch=random.sample(self.memory,batch_size)
        for experience in mini_batch:
            state,action,reward,next_state,done=experience
            if not done:
                target=reward+self.gamma*np.amax(self.model.predict(next_state)[0])
            else:
                target=reward
            target_f=self.model.predict(state)
            target_f[0][action]=target
            self.model.fit(state,target_f,epochs=1,verbose=0)
        if self.epsilon>self.epsilon_min:
            self.epsilon*=self.epsilon_decay

In [5]:
n_episodes=1000
action_size=3
state_size=2
batch_size=32

In [6]:
import warnings
warnings.filterwarnings('ignore')
agent=Agent(action_size,state_size)

W1123 18:16:15.745463 15876 deprecation_wrapper.py:119] From c:\users\ayush\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1123 18:16:15.769459 15876 deprecation_wrapper.py:119] From c:\users\ayush\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1123 18:16:15.774461 15876 deprecation_wrapper.py:119] From c:\users\ayush\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1123 18:16:15.867444 15876 deprecation_wrapper.py:119] From c:\users\ayush\appdata\local\programs\python\python36\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.

In [7]:
for e in range(n_episodes):
    state=env.reset()
    state=np.reshape(state,[1,state_size])
    for t in range(8000):
        env.render()
        action=agent.act(state)
        next_state,reward,done,other_info=env.step(action)
        reward=reward if not done else -10 # ye q h ye line !!!!!!!!!!1
        next_state=np.reshape(next_state,[1,state_size])
        agent.remember(state,action,reward,next_state,done)
        state=next_state
        if  done:
            print("Game over: {}/{} High Score: {} Exploration_rate:{:.2}".format(e,n_episodes,t,agent.epsilon))
            break
    if len(agent.memory)>batch_size:
        agent.train(batch_size)
print('Deep Q Learner model Trained !!!!')
env.close()

W1123 18:16:20.055752 15876 deprecation_wrapper.py:119] From c:\users\ayush\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:2741: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W1123 18:16:20.058751 15876 deprecation_wrapper.py:119] From c:\users\ayush\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



Game over: 0/1000 High Score: 199 Exploration_rate:1.0
Game over: 1/1000 High Score: 199 Exploration_rate:0.99
Game over: 2/1000 High Score: 199 Exploration_rate:0.99
Game over: 3/1000 High Score: 199 Exploration_rate:0.99
Game over: 4/1000 High Score: 199 Exploration_rate:0.98
Game over: 5/1000 High Score: 199 Exploration_rate:0.98
Game over: 6/1000 High Score: 199 Exploration_rate:0.97
Game over: 7/1000 High Score: 199 Exploration_rate:0.97
Game over: 8/1000 High Score: 199 Exploration_rate:0.96
Game over: 9/1000 High Score: 199 Exploration_rate:0.96
Game over: 10/1000 High Score: 199 Exploration_rate:0.95
Game over: 11/1000 High Score: 199 Exploration_rate:0.95
Game over: 12/1000 High Score: 199 Exploration_rate:0.94
Game over: 13/1000 High Score: 199 Exploration_rate:0.94
Game over: 14/1000 High Score: 199 Exploration_rate:0.93
Game over: 15/1000 High Score: 199 Exploration_rate:0.93
Game over: 16/1000 High Score: 199 Exploration_rate:0.92
Game over: 17/1000 High Score: 199 Explora

Game over: 144/1000 High Score: 199 Exploration_rate:0.49
Game over: 145/1000 High Score: 199 Exploration_rate:0.48
Game over: 146/1000 High Score: 199 Exploration_rate:0.48
Game over: 147/1000 High Score: 199 Exploration_rate:0.48
Game over: 148/1000 High Score: 199 Exploration_rate:0.48
Game over: 149/1000 High Score: 199 Exploration_rate:0.47
Game over: 150/1000 High Score: 199 Exploration_rate:0.47
Game over: 151/1000 High Score: 199 Exploration_rate:0.47
Game over: 152/1000 High Score: 199 Exploration_rate:0.47
Game over: 153/1000 High Score: 199 Exploration_rate:0.46
Game over: 154/1000 High Score: 199 Exploration_rate:0.46
Game over: 155/1000 High Score: 199 Exploration_rate:0.46
Game over: 156/1000 High Score: 199 Exploration_rate:0.46
Game over: 157/1000 High Score: 199 Exploration_rate:0.46
Game over: 158/1000 High Score: 199 Exploration_rate:0.45
Game over: 159/1000 High Score: 199 Exploration_rate:0.45
Game over: 160/1000 High Score: 199 Exploration_rate:0.45
Game over: 161

Game over: 286/1000 High Score: 199 Exploration_rate:0.24
Game over: 287/1000 High Score: 199 Exploration_rate:0.24
Game over: 288/1000 High Score: 199 Exploration_rate:0.24
Game over: 289/1000 High Score: 199 Exploration_rate:0.23
Game over: 290/1000 High Score: 199 Exploration_rate:0.23
Game over: 291/1000 High Score: 199 Exploration_rate:0.23
Game over: 292/1000 High Score: 199 Exploration_rate:0.23
Game over: 293/1000 High Score: 199 Exploration_rate:0.23
Game over: 294/1000 High Score: 199 Exploration_rate:0.23
Game over: 295/1000 High Score: 199 Exploration_rate:0.23
Game over: 296/1000 High Score: 199 Exploration_rate:0.23
Game over: 297/1000 High Score: 199 Exploration_rate:0.23
Game over: 298/1000 High Score: 199 Exploration_rate:0.22
Game over: 299/1000 High Score: 199 Exploration_rate:0.22
Game over: 300/1000 High Score: 199 Exploration_rate:0.22
Game over: 301/1000 High Score: 199 Exploration_rate:0.22
Game over: 302/1000 High Score: 199 Exploration_rate:0.22
Game over: 303

Game over: 428/1000 High Score: 199 Exploration_rate:0.12
Game over: 429/1000 High Score: 199 Exploration_rate:0.12
Game over: 430/1000 High Score: 199 Exploration_rate:0.12
Game over: 431/1000 High Score: 199 Exploration_rate:0.12
Game over: 432/1000 High Score: 199 Exploration_rate:0.11
Game over: 433/1000 High Score: 199 Exploration_rate:0.11
Game over: 434/1000 High Score: 199 Exploration_rate:0.11
Game over: 435/1000 High Score: 199 Exploration_rate:0.11
Game over: 436/1000 High Score: 199 Exploration_rate:0.11
Game over: 437/1000 High Score: 199 Exploration_rate:0.11
Game over: 438/1000 High Score: 199 Exploration_rate:0.11
Game over: 439/1000 High Score: 199 Exploration_rate:0.11
Game over: 440/1000 High Score: 199 Exploration_rate:0.11
Game over: 441/1000 High Score: 199 Exploration_rate:0.11
Game over: 442/1000 High Score: 199 Exploration_rate:0.11
Game over: 443/1000 High Score: 199 Exploration_rate:0.11
Game over: 444/1000 High Score: 199 Exploration_rate:0.11
Game over: 445

Game over: 568/1000 High Score: 199 Exploration_rate:0.058
Game over: 569/1000 High Score: 199 Exploration_rate:0.058
Game over: 570/1000 High Score: 199 Exploration_rate:0.057
Game over: 571/1000 High Score: 199 Exploration_rate:0.057
Game over: 572/1000 High Score: 199 Exploration_rate:0.057
Game over: 573/1000 High Score: 199 Exploration_rate:0.057
Game over: 574/1000 High Score: 199 Exploration_rate:0.056
Game over: 575/1000 High Score: 199 Exploration_rate:0.056
Game over: 576/1000 High Score: 199 Exploration_rate:0.056
Game over: 577/1000 High Score: 199 Exploration_rate:0.055
Game over: 578/1000 High Score: 199 Exploration_rate:0.055
Game over: 579/1000 High Score: 199 Exploration_rate:0.055
Game over: 580/1000 High Score: 199 Exploration_rate:0.055
Game over: 581/1000 High Score: 199 Exploration_rate:0.054
Game over: 582/1000 High Score: 199 Exploration_rate:0.054
Game over: 583/1000 High Score: 199 Exploration_rate:0.054
Game over: 584/1000 High Score: 199 Exploration_rate:0.0

Game over: 708/1000 High Score: 199 Exploration_rate:0.029
Game over: 709/1000 High Score: 199 Exploration_rate:0.029
Game over: 710/1000 High Score: 199 Exploration_rate:0.028
Game over: 711/1000 High Score: 199 Exploration_rate:0.028
Game over: 712/1000 High Score: 199 Exploration_rate:0.028
Game over: 713/1000 High Score: 199 Exploration_rate:0.028
Game over: 714/1000 High Score: 199 Exploration_rate:0.028
Game over: 715/1000 High Score: 199 Exploration_rate:0.028
Game over: 716/1000 High Score: 199 Exploration_rate:0.028
Game over: 717/1000 High Score: 199 Exploration_rate:0.027
Game over: 718/1000 High Score: 199 Exploration_rate:0.027
Game over: 719/1000 High Score: 199 Exploration_rate:0.027
Game over: 720/1000 High Score: 199 Exploration_rate:0.027
Game over: 721/1000 High Score: 199 Exploration_rate:0.027
Game over: 722/1000 High Score: 199 Exploration_rate:0.027
Game over: 723/1000 High Score: 199 Exploration_rate:0.027
Game over: 724/1000 High Score: 199 Exploration_rate:0.0

Game over: 848/1000 High Score: 199 Exploration_rate:0.014
Game over: 849/1000 High Score: 199 Exploration_rate:0.014
Game over: 850/1000 High Score: 199 Exploration_rate:0.014
Game over: 851/1000 High Score: 199 Exploration_rate:0.014
Game over: 852/1000 High Score: 199 Exploration_rate:0.014
Game over: 853/1000 High Score: 199 Exploration_rate:0.014
Game over: 854/1000 High Score: 199 Exploration_rate:0.014
Game over: 855/1000 High Score: 199 Exploration_rate:0.014
Game over: 856/1000 High Score: 199 Exploration_rate:0.014
Game over: 857/1000 High Score: 199 Exploration_rate:0.014
Game over: 858/1000 High Score: 199 Exploration_rate:0.014
Game over: 859/1000 High Score: 199 Exploration_rate:0.013
Game over: 860/1000 High Score: 199 Exploration_rate:0.013
Game over: 861/1000 High Score: 199 Exploration_rate:0.013
Game over: 862/1000 High Score: 199 Exploration_rate:0.013
Game over: 863/1000 High Score: 199 Exploration_rate:0.013
Game over: 864/1000 High Score: 199 Exploration_rate:0.0

Game over: 989/1000 High Score: 199 Exploration_rate:0.01
Game over: 990/1000 High Score: 199 Exploration_rate:0.01
Game over: 991/1000 High Score: 199 Exploration_rate:0.01
Game over: 992/1000 High Score: 199 Exploration_rate:0.01
Game over: 993/1000 High Score: 199 Exploration_rate:0.01
Game over: 994/1000 High Score: 199 Exploration_rate:0.01
Game over: 995/1000 High Score: 199 Exploration_rate:0.01
Game over: 996/1000 High Score: 199 Exploration_rate:0.01
Game over: 997/1000 High Score: 199 Exploration_rate:0.01
Game over: 998/1000 High Score: 199 Exploration_rate:0.01
Game over: 999/1000 High Score: 199 Exploration_rate:0.01
Deep Q Learner model Trained !!!!
